In [1]:
import spacy
from gensim.models import KeyedVectors

# glove = KeyedVectors.load_word2vec_format('../glove.6B.300d.w2vformat.txt')
# w2v = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)
# nlp = spacy.load("../model_wiki")
sim = 'elmo-ori/'
t = '-elmo_ori'

In [2]:
from scipy import spatial

def cosine(x, y):
    dataSetI = x
    dataSetII = y
    return 1 - spatial.distance.cosine(dataSetI, dataSetII)

import string
def rem_punct(word):
    return word.translate(str.maketrans('', '', string.punctuation))

In [3]:
import pickle

def load_category(category):
    pickle_in = open("elmo_category/"+ sim + category +".pickle", "rb")
    return pickle.load(pickle_in)

def load_data(id_file):
    pickle_in = open("elmo_restaurant/absa16/"+ str(id_file).replace(':','_') +".pickle", "rb")
    return pickle.load(pickle_in)[0]

def load_kata(id_file):
    pickle_in = open("elmo_restaurant/all_terms/"+ str(id_file).replace('/','_').replace('.', '_') +".pickle", "rb")
    return pickle.load(pickle_in)

In [4]:
import numpy as np

def similarity(category, word):
    res = []
    for cat in category:
        res.append(cosine(cat, word))
    return np.mean(res)

In [5]:
def sim_result(word):
    c1 = similarity(ambience, word)
    c2 = similarity(food, word)
    c3 = similarity(service, word)
    c4 = similarity(value, word)
    return c1, c2, c3, c4

In [6]:
def cat_str(a, b, c, d):
    if a > b and a > c and a > d:
        return 1
    elif b > a and b > c and b > d:
        return 2
    elif c > b and c > a and c > d:
        return 3
    elif d > b and d > c and d > a:
        return 4
    else:
        return 5

In [7]:
def str_cat(num):
    pred =''
    if num == 1:
        pred = 'AMBIENCE'
    elif num == 2:
        pred = 'FOOD'
    elif num == 3:
        pred = 'SERVICE'
    elif num == 4:
        pred = 'PRICES'
    return pred

In [8]:
# service

In [9]:
ambience = load_category('ambience')
food = load_category('food')
service = load_category('service')
value = load_category('price')

import pandas as pd
from tqdm import tqdm_notebook

def run(tipe, tipe_keluar, where):
    ambience = load_category('ambience')
    food = load_category('food')
    service = load_category('service')
    value = load_category('price')
    
    df = pd.read_csv('Results/' + where + tipe +'.csv')
    sf = pd.DataFrame(columns=['id', 'review', 'target', 'label', 'predict', 'term', 'polarity'])
    for index in tqdm_notebook(range(len(df))):
        pred= ''
        pred_mult=[]
        if type(df['term'][index]) != float:
            terms = df['term'][index].lower().split('|')
            target = df['target'][index]
            id_file = df['id'][index]
            label = df['category'][index]
            review = df['review'][index]
            polarity = df['polarity'][index]

            vector = load_data(id_file)
            tokenized = df['review'][index].lower().replace('  ', ' ').strip().split(' ')

            for term in terms:
                if term == '':
                    continue
                i = 0
                for token in tokenized:
                    if term in token:
                        if sim == 'elmo-ori/':
                            a, b, c, d = sim_result(vector[i])
#                         elif sim == 'elmo-ori/':
#                             a, b, c, d = sim_result(load_kata(term))
                        elif sim == 'fasttext/':
                            a, b, c, d = sim_result(nlp(rem_punct(token)).vector)
                        elif sim == 'w2v/':
                            try:
                                a, b, c, d = sim_result(w2v[rem_punct(token)])
                            except:
                                a, b, c, d = 0, 0, 0, 0
                        elif sim == 'glove/':
                            try:
                                a, b, c, d = sim_result(glove[rem_punct(token)])
                            except:
                                a, b, c, d = 0, 0, 0, 0

                        x = cat_str(a, b, c, d)
                        pred_mult.append(str_cat(x))
                        break
                    i += 1
        else:
            terms = ''
            target = df['target'][index]
            id_file = df['id'][index]
            label = df['category'][index]
            polarity = df['polarity'][index]
            review = df['review'][index]
            
        sf = sf.append({'id': id_file,
                        'review': review,
                        'target': target,
                        'label': label,
                        'predict': '|'.join(pred_mult),
                        'term': '|'.join(terms),
                        'polarity': polarity
                       }, ignore_index=True)

        sf.to_csv("Results/Categorization/"+ tipe +tipe_keluar+".csv")
        sf.to_excel("Results/Categorization/"+ tipe +tipe_keluar+".xlsx")

In [ ]:
# run('plsa-partial', '-elmo_ori', 'add_noun/')

In [ ]:
# run('lda2vec-partial', '-elmo_ori', 'add_noun/')

In [ ]:
# run('lda-partial', '-elmo_ori', 'add_noun/')

In [ ]:
# run('dp-partial', '-elmo_ori', 'add_noun/')

In [ ]:
# run('plsa', '-elmo_ori', 'Aspect Terms Extraction/')

In [ ]:
# run('lda2vec', '-elmo_ori', 'Aspect Terms Extraction/')

In [ ]:
# run('lda', '-elmo_ori', 'Aspect Terms Extraction/')

In [ ]:
# run('dp', '-elmo_ori', 'Aspect Terms Extraction/')

In [10]:
run('plsa-partial-ner', t, 'Filter/')

HBox(children=(IntProgress(value=0, max=462), HTML(value='')))

In [11]:
run('lda2vec-partial-ner', t, 'Filter/')

HBox(children=(IntProgress(value=0, max=462), HTML(value='')))

In [12]:
run('lda-partial-ner', t, 'Filter/')

HBox(children=(IntProgress(value=0, max=462), HTML(value='')))

In [13]:
run('dp-partial-ner', t, 'Filter/')

HBox(children=(IntProgress(value=0, max=462), HTML(value='')))